In [20]:
import math
import shutil
import random
import argparse
import numpy as np
import cv2, imageio
import os, subprocess
from _gradient import get_gradient
import time

#### Image gradients are a fundamental building block of many computer vision and image processing routines.
##### -We use gradients for detecting edges in images, which allows us to find contours and outlines of objects in images
##### -We use them as inputs for quantifying images through feature extraction



#### Let's first disrupt the order of coloring

In [21]:
def draw_order(h, w, scale):
    order = []
    for i in range(0, h, scale):
        for j in range(0, w, scale):
            y = random.randint(-scale // 2, scale // 2) + i
            x = random.randint(-scale // 2, scale // 2) + j
            order.append((y % h, x % w))
    return order


In [34]:
#Parameter you can choose from 
brush_width=2 #The size of the brush
path='lake.jpeg' #The path of the picss
gradient='sobel' # The type of the artstyle you want to choose


## First Let's transfer pictures into numpy arrays

In [35]:
img_path = path.rsplit(".", -1) 

In [23]:
img = cv2.imread(path)

In [24]:
img.shape 
#1050 rows,1680 columns and 3 RGB colors inside

(1050, 1680, 3)

#### the main application of image gradients lies within edge detection.
##### edge detection is the process of finding edges in an image, which reveals structural information regarding the objects in an image.

In [25]:
r = 2 * int(img.shape[0] / 50) + 1
Gx, Gy = get_gradient(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (r, r), gradient)


#### Based on the edge dection, we can draw the ellipse as the brush

In [26]:
Gh = np.sqrt(np.sqrt(np.square(Gx) + np.square(Gy)))    # Length of the ellipse
Ga = (np.arctan2(Gy, Gx) / np.pi) * 180 + 90            # Angle of the ellipse


#### First we can blur thee images and make it more artlistic

In [27]:
canvas = cv2.medianBlur(img, 11)    # Make the image artlistic
result=[]

In [28]:
order = draw_order(img.shape[0], img.shape[1], scale=brush_width*2)
colors = np.array(img, dtype=np.float)

<ipython-input-28-f909b52cd348>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  colors = np.array(img, dtype=np.float)


In [10]:
#### 

In [29]:
for i, (y, x) in enumerate(order):
    length = int(round(brush_width + brush_width * Gh[y, x]))
    color = colors[y,x]
    cv2.ellipse(canvas, (x, y), (length, brush_width), Ga[y, x], 0, 360, color, -1, cv2.LINE_AA)

result.append(canvas)

In [30]:
result[0]

array([[[192, 109,  48],
        [192, 109,  48],
        [192, 109,  48],
        ...,
        [196, 165, 132],
        [196, 165, 132],
        [196, 165, 132]],

       [[192, 109,  48],
        [192, 109,  48],
        [194, 111,  50],
        ...,
        [196, 165, 134],
        [196, 165, 132],
        [196, 165, 132]],

       [[193, 110,  49],
        [193, 110,  49],
        [194, 111,  50],
        ...,
        [196, 167, 136],
        [196, 167, 136],
        [196, 165, 132]],

       ...,

       [[ 92,  82,  65],
        [ 92,  82,  65],
        [ 92,  82,  65],
        ...,
        [ 85,  75,  65],
        [ 85,  75,  65],
        [ 85,  75,  65]],

       [[ 92,  82,  65],
        [ 92,  82,  65],
        [ 92,  82,  65],
        ...,
        [ 85,  75,  65],
        [ 85,  75,  65],
        [ 85,  75,  65]],

       [[ 92,  82,  65],
        [ 92,  82,  65],
        [ 92,  82,  65],
        ...,
        [ 85,  75,  65],
        [ 85,  75,  65],
        [ 85,  75,  65]]

In [37]:
print("\nOutput the result")
output_path = img_path[0]+'_result.'+img_path[1]
print(output_path)
if img_path[1] == 'gif':
    c, images = 0, []
    for canva in result:
        cv2.imwrite(img_path[0]+'/r-'+str(c)+'.jpg', canva); c += 1
    for i in range(c-1): images.append(imageio.imread(img_path[0]+'/r-'+str(i)+'.jpg'))
    from PIL import Image
    imageio.mimsave(output_path, images, duration=float(Image.open(args.path).info['duration']) / 1000)
    shutil.rmtree(img_path[0], ignore_errors=True)
else:
    cv2.imwrite(output_path, result[0])


Output the result
lake_result.jpeg
